In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import keras_tuner as kt

In [2]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

# train_data.head()
train_data.tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.7,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [3]:
train_data = train_data.drop(labels=["Cabin", "Name", "Ticket", "PassengerId"], axis=1)
# train_data = train_data.dropna() # i just changed both by hand to include average age when there was no other data availible

test_data = test_data.drop(labels=["Cabin", "Name", "Ticket"], axis=1)
test_ids = test_data.pop("PassengerId")
# test_data = test_data.dropna()

train_data = train_data.replace("male", 1).replace("female", 0).replace("S", 0).replace("C", 1).replace("Q", 2)
test_data = test_data.replace("male", 1).replace("female", 0).replace("S", 0).replace("C", 1).replace("Q", 2)
train_data.tail()

print(train_data.isna().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64


In [4]:
x_train = train_data.copy()
# have to split data lol
y_train = x_train.pop("Survived")
x_train.tail()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
886,2,1,27.0,0,0,13.00,0.0
887,1,0,19.0,0,0,30.00,0.0
888,3,0,28.7,1,2,23.45,0.0
889,1,1,26.0,0,0,30.00,1.0
890,3,1,32.0,0,0,7.75,2.0


In [5]:
x_train.describe().transpose()[["mean", "std"]]

,mean,std
Pclass,2.308642,0.836071
Sex,0.647587,0.477990
Age,29.765062,13.008481
SibSp,0.523008,1.102743
Parch,0.381594,0.806057
Fare,32.204208,49.693429
Embarked,0.362205,0.636157


In [6]:
x_train = np.asarray(x_train).astype('float32')
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(x_train))
normalizer.mean.numpy()

2022-06-28 21:53:48.500788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


array([[ 2.308642  ,  0.64758706, 29.76506   ,  0.5230078 ,  0.38159373,
        32.20421   ,         nan]], dtype=float32)

In [7]:
def model_builder(hp):
    global learning_rate
    learning_rate = hp.Choice('learning_rate', values=[5e-1, 1e-1, 1e-2, 1e-3, 1e-4])
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int("units2", min_value=32, max_value=512, step=32)
    hp_units3 = hp.Int("units3", min_value=32, max_value=512, step=32)
    hp_dropout = hp.Float("dropout", min_value=0, max_value=0.5)
    
    model = keras.Sequential([
        normalizer, 
        keras.layers.Dense(hp_units, kernel_regularizer=keras.regularizers.l2(0.001), activation="relu"),
        keras.layers.Dropout(hp_dropout),
        keras.layers.Dense(hp_units2, kernel_regularizer=keras.regularizers.l2(0.001), activation="relu"),
        keras.layers.Dropout(hp_dropout),
        keras.layers.Dense(hp_units3, kernel_regularizer=keras.regularizers.l2(0.001), activation="relu"),
        keras.layers.Dropout(hp_dropout),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=tf.metrics.BinaryAccuracy(threshold=0.0))
    return model
# model_builder(normalizer).summary()

probably should add early stopping

In [8]:
tuner = kt.Hyperband(model_builder, objective="val_binary_accuracy", max_epochs=100, factor=3, directory="my_dir", project_name="kaggle_titanic")
tuner.search(x_train, y_train, epochs=50, validation_split=0.2)

Trial 200 Complete [00h 00m 03s]
val_binary_accuracy: 0.6424580812454224

Best val_binary_accuracy So Far: 0.6424580812454224
Total elapsed time: 00h 08m 21s

Search: Running Trial #201

Value             |Best Value So Far |Hyperparameter
0.1               |0.0001            |learning_rate
256               |64                |units
160               |160               |units2
480               |384               |units3
0.34345           |0.44654           |dropout
12                |2                 |tuner/epochs
4                 |0                 |tuner/initial_epoch
3                 |4                 |tuner/bracket
1                 |0                 |tuner/round
0159              |None              |tuner/trial_id

Epoch 5/12


KeyboardInterrupt: 

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("The hyperparameter search is complete.")

The hyperparameter search is complete.


In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=500, validation_split=0.2, verbose=0)
val_acc_per_epoch = history.history['val_binary_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


KeyError: 'val_binary_accuracy'

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/436
23/23 [==============================] - 1s 5ms/step - loss: 1.1561 - binary_accuracy: 0.7598
Epoch 2/436
23/23 [==============================] - 0s 5ms/step - loss: 0.9208 - binary_accuracy: 0.7992
Epoch 3/436
23/23 [==============================] - 0s 5ms/step - loss: 0.7885 - binary_accuracy: 0.8272
Epoch 4/436
23/23 [==============================] - 0s 6ms/step - loss: 0.7131 - binary_accuracy: 0.8188
Epoch 5/436
23/23 [==============================] - 0s 6ms/step - loss: 0.6529 - binary_accuracy: 0.8244
Epoch 6/436
23/23 [==============================] - 0s 8ms/step - loss: 0.6065 - binary_accuracy: 0.8315
Epoch 7/436
23/23 [==============================] - 0s 5ms/step - loss: 0.5869 - binary_accuracy: 0.8357
Epoch 8/436
23/23 [==============================] - 0s 5ms/step - loss: 0.5490 - binary_accuracy: 0.8343
Epoch 9/436
23/23 [==============================] - 0s 5ms/step - loss: 0.5381 - binary_accuracy: 0.8287
Epoch 10/436
23/23 [==========================

In [ ]:
export_model = tf.keras.Sequential([
  hypermodel,
  keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['val_binary_accuracy']
)

In [ ]:
results = export_model.predict(test_data)
with open("ai_submission.csv", "w") as f:
    f.write("PassengerId,Survived\n")
    for i, val in enumerate(results):
        f.write(f"{test_ids[i]},{round(val[0])}\n")


11/11 [==============================] - 0s 2ms/step
     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    1  34.5      0      0    7.8292         2
1         3    0  47.0      1      0    7.0000         0
2         2    1  62.0      0      0    9.6875         2
3         3    1  27.0      0      0    8.6625         0
4         3    0  22.0      1      1   12.2875         0
..      ...  ...   ...    ...    ...       ...       ...
409       3    0   3.0      1      1   13.7750         0
411       1    0  37.0      1      0   90.0000         2
412       3    0  28.0      0      0    7.7750         0
414       1    0  39.0      0      0  108.9000         1
415       3    1  38.5      0      0    7.2500         0

[331 rows x 7 columns]
